In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the btuB gene
model0['lb_list']['EX_cbl1_e ']=0
model0['lb_list']['EX_cbl1_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                                    if i == 0:
                                        # gene upregulation for Fe2+ intake
                                        if 'EX_fe2_e' in rea:
                                            public_reaction_ub_list[i][rea][0, m] = public_reaction_ub_list[i][rea][0, m] * 1.2
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([20, 13,  6, 20, 18,  4, 17, 12,  7, 13,  4,  2, 11,  9, 16, 14,  5,
       11, 17,  6, 19, 17, 15, 19, 12,  4,  2,  8,  4,  5]), 1: array([ 7, 11,  8, 19,  2, 18, 15,  5, 16,  6,  2, 17, 16, 10, 10,  7, 15,
        5, 19, 13,  7, 14,  9,  7, 13, 13,  9,  1,  7, 19])}
1
strain_number:  0 12.766666666666667
strain_various:  0 7.0222661742647015
strain_number:  1 10.666666666666666
strain_various:  1 5.274677451960663
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([20, 13,  6, 20, 18,  4, 17, 12,  7, 13,  4,  2, 11,  9, 16, 14,  5,
       11, 17,  6, 19, 17, 15, 19, 12,  4,  2,  8,  4,  5]), 1: array([ 7, 11,  8, 19,  2, 18, 15,  5, 16,  6,  2, 17, 16, 10, 10,  7, 15,
        5, 19, 13,  7, 14,  9,  7, 13, 13,  9,  1,  7, 19])}
_2:  {0: [0.6399142351471028, 0.2857391744693639], 1: [0.6399142351471028, 0.2857391744693639], 2: [0.6399142351471028, 0.2857391744693639], 3: [0.6399142351471028, 0.2857391744693639], 4: [0.6399142351471028, 0.2857391744693639],

glc__D_e_1:  -18.408367165422504
glc__D_e_1:  -6.898097374941978
glc__D_e_1:  -17.85089626992802
glc__D_e_1:  -0.7745255712292014
glc__D_e_1:  -14.140913396019027
glc__D_e_1:  -18.748619770746426
glc__D_e_1:  -13.678659704850382
glc__D_e_1:  -13.226629486234938
glc__D_e_1:  -0.08501443799456965
glc__D_e_1:  -0.7881764967003085
glc__D_e_1:  -20.38371297057917
glc__D_e_1:  -26.776370740952544
glc__D_e_1:  -0.49314064447180783
glc__D_e_1:  -16.86024369326757
glc__D_e_1:  -1.8668063643258783
glc__D_e_1:  -13.14552770335342
glc__D_e_1:  -0.6725867035455622
glc__D_e_1:  -0.07890529872518659
glc__D_e_1:  -26.622637134464874
glc__D_e_1:  -25.0533347926095
glc__D_e_1:  -0.3045812624257529
glc__D_e_1:  -0.44778963177673115
glc__D_e_1:  -13.09513074379477
glc__D_e_1:  -18.31645390226671
glc__D_e_1:  -8.57834911988207
glc__D_e_1:  -17.638601783643853
glc__D_e_1:  -8.655910517577807
glc__D_e_1:  -15.578015566106899
glc__D_e_1:  -9.924939201096915
strain_number:  0 35.7
strain_various:  0 19.9668892

glc__D_e_1:  -0.16569047785527347
glc__D_e_1:  -0.16302616551129678
glc__D_e_1:  -0.9297156966228985
glc__D_e_1:  -1.513269195372032
glc__D_e_1:  -2.916166450678305
glc__D_e_1:  -2.80751653438777
glc__D_e_1:  -3.342711031503853
glc__D_e_1:  -2.435641622399984
glc__D_e_1:  -1.3089332277291912
glc__D_e_1:  -0.8063355340256673
glc__D_e_1:  -1.4707937861549087
glc__D_e_1:  -0.24938435025189182
glc__D_e_1:  -1.016731252860445
glc__D_e_1:  -1.7020872179739148
glc__D_e_1:  -6.2476538880656465
glc__D_e_1:  -2.6821611043132134
glc__D_e_1:  -20.66606106443724
glc__D_e_1:  -33.738380875545694
glc__D_e_1:  -32.62578142511562
strain_number:  0 5.7
strain_various:  0 10.596068453283353
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 661.6361337321089, 339.8613688942637, 365.49414313756427, 398.3012765743029, 444.06535417834607, 418.918039423566, 472.6340989106443, 528.5112164312379, 435.39693777721635, 430.5327112126517, 477.3400767363744, 414.0369198177722, 3

In [7]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([11, 10, 15,  6, 14, 10, 11, 11, 12,  2, 10, 15,  8, 11, 13,  1, 15,
        5,  8,  1,  4, 13,  4, 11,  8, 16,  2,  7,  2, 12]), 1: array([ 9,  4,  7,  4, 17,  3,  9, 20, 10,  8,  8,  2, 16,  8,  4,  5, 11,
        1,  1,  6,  3,  6, 19,  3,  4, 17,  6, 10, 14,  8])}
1
strain_number:  0 10.4
strain_various:  0 5.4808758424178885
strain_number:  1 8.1
strain_various:  1 5.293706955747865
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([11, 10, 15,  6, 14, 10, 11, 11, 12,  2, 10, 15,  8, 11, 13,  1, 15,
        5,  8,  1,  4, 13,  4, 11,  8, 16,  2,  7,  2, 12]), 1: array([ 9,  4,  7,  4, 17,  3,  9, 20, 10,  8,  8,  2, 16,  8,  4,  5, 11,
        1,  1,  6,  3,  6, 19,  3,  4, 17,  6, 10, 14,  8])}
_2:  {0: [0.6399142351471028, 0.2857391744693639], 1: [0.6399142351471028, 0.2857391744693639], 2: [0.6399142351471028, 0.2857391744693639], 3: [0.6399142351471028, 0.2857391744693639], 4: [0.6399142351471028, 0.2857391744693639], 5: [0.6399142351471028, 0.28

glc__D_e_1:  -19.519053962973764
glc__D_e_1:  -16.146177431357206
glc__D_e_1:  -5.520197428423712
glc__D_e_1:  -19.526294851994972
glc__D_e_1:  -5.007262052157226
glc__D_e_1:  -11.839835295917958
glc__D_e_1:  -25.68657169818798
glc__D_e_1:  -16.98064527218276
glc__D_e_1:  -22.321231530685136
glc__D_e_1:  -18.502180397308237
glc__D_e_1:  -33.5126370223935
glc__D_e_1:  -18.96936352029133
glc__D_e_1:  -15.45036824473138
glc__D_e_1:  -11.849995680508119
strain_number:  0 39.8
strain_various:  0 18.49936935862049
strain_number:  1 0.03333333333333333
strain_various:  1 0.1795054935711501
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 796.2456677739185, 342.5580362619837, 423.668238559686, 549.3952065478381, 466.65037046287057, 388.0860897874056, 418.89827065349516, 459.6502759237667, 433.48499235320867, 415.3681935952531, 357.36325198571603, 403.98647465429354, 359.7329879128087, 253.04897925932616, 232.82939472751866]
slip_r:  232.82939472751866
17
glc__D_e_1:  -9.189410638465214
glc__D_e_1:  

glc__D_e_1:  -7.123821786121788
glc__D_e_1:  -13.352926497354133
glc__D_e_1:  -11.005978895926557
glc__D_e_1:  -6.3477841924362535
glc__D_e_1:  -9.807962898751843
glc__D_e_1:  -3.329724422289873
glc__D_e_1:  -6.831451275946287
glc__D_e_1:  -3.054802528946404
glc__D_e_1:  -7.6431135944421795
glc__D_e_1:  -1.84848312202028
glc__D_e_1:  -3.1789240639264142
strain_number:  0 4.7
strain_various:  0 4.768997099321128
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 796.2456677739185, 342.5580362619837, 423.668238559686, 549.3952065478381, 466.65037046287057, 388.0860897874056, 418.89827065349516, 459.6502759237667, 433.48499235320867, 415.3681935952531, 357.36325198571603, 403.98647465429354, 359.7329879128087, 253.04897925932616, 232.82939472751866, 179.5810323868896, 147.53784856818098, 178.40384701416238, 182.3626342193276, 220.8556913147317, 214.13794903164347]
slip_r:  214.13794903164347
23
glc__D_e_1:  -0.23981934170068298
glc__D_e_1:  -0.98388255

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([13, 10, 10,  2, 19,  1,  1,  7, 13, 17, 11, 10, 10, 12,  4,  9, 19,
        1, 13, 18,  8, 20, 15,  6,  6,  8,  9,  9, 14, 13]), 1: array([16,  5, 11,  6,  5, 14,  6,  9, 16, 20, 17,  6,  4, 20, 16, 12,  6,
        8, 18,  8,  7, 20,  5,  2, 16, 18,  8, 11,  9, 19])}
1
strain_number:  0 11.9
strain_various:  0 6.33692880397647
strain_number:  1 11.266666666666667
strain_various:  1 5.591263025669802
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([13, 10, 10,  2, 19,  1,  1,  7, 13, 17, 11, 10, 10, 12,  4,  9, 19,
        1, 13, 18,  8, 20, 15,  6,  6,  8,  9,  9, 14, 13]), 1: array([16,  5, 11,  6,  5, 14,  6,  9, 16, 20, 17,  6,  4, 20, 16, 12,  6,
        8, 18,  8,  7, 20,  5,  2, 16, 18,  8, 11,  9, 19])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988

glc__D_e_1:  -11.992361579432472
glc__D_e_1:  -1.8356100276850515
glc__D_e_1:  -14.57850764232367
strain_number:  0 44.03333333333333
strain_various:  0 15.477905399489801
strain_number:  1 2.033333333333333
strain_various:  1 1.4255603186895398
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 688.3097728509879, 346.6665531836584, 397.0860915400389, 496.4667481641166, 541.2559623125412, 552.5565443016819, 555.9860716421474, 556.7895194049507, 462.41749662640603, 439.5163938319661, 450.2169091067045, 412.0540991018955, 361.5650083105654]
slip_r:  361.5650083105654
15
glc__D_e_1:  -5.478597525511191
glc__D_e_1:  -5.440061875207796
glc__D_e_1:  -0.027349075117830246
glc__D_e_1:  -27.21224566103861
glc__D_e_1:  -23.624460303589817
glc__D_e_1:  -1.3049350547974434
glc__D_e_1:  -0.7327863609068278
glc__D_e_1:  -16.078312120717563
glc__D_e_1:  -22.74413172236889
glc__D_e_1:  -0.4246064735765174
glc__D_e_1:  -0.5714396445528782
glc__D_e_1:  -18.518698965016707
glc__D_e_1:  -19.63305620750162
glc__D_

glc__D_e_1:  -4.768943671202495
glc__D_e_1:  -1.6650846402130406
glc__D_e_1:  -7.8601152829449745
glc__D_e_1:  -8.337085122688825
glc__D_e_1:  -4.115966380021858
glc__D_e_1:  -3.497575947459345
glc__D_e_1:  -7.089367934513618
glc__D_e_1:  -2.2464746984960016
glc__D_e_1:  -4.127009179070504
glc__D_e_1:  -3.294326377342489
glc__D_e_1:  -1.9485717949301007
glc__D_e_1:  -3.4054069981184822
glc__D_e_1:  -3.9122666532163453
glc__D_e_1:  -1.350592812387454
glc__D_e_1:  -0.8864653750394451
glc__D_e_1:  -3.3145255459774403
glc__D_e_1:  -1.2045385329194689
glc__D_e_1:  -0.8836283367158188
glc__D_e_1:  -0.8841505973913919
glc__D_e_1:  -0.8740875539424902
glc__D_e_1:  -0.1233360081091388
glc__D_e_1:  -0.3708223320754005
glc__D_e_1:  -0.940962679710468
glc__D_e_1:  -1.348411727209771
glc__D_e_1:  -1.383750991219193
glc__D_e_1:  -0.9839400651704742
glc__D_e_1:  -0.9745535633713648
glc__D_e_1:  -1.8374190051245685
glc__D_e_1:  -1.1294826937382254
strain_number:  0 4.1
strain_various:  0 3.21817753808

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 9, 17, 16,  3, 13, 11, 20, 15, 13,  2, 17, 19, 12, 11, 14, 17,  1,
       16, 12, 17, 16, 16,  4,  6, 16,  9,  5, 10, 14, 17]), 1: array([11, 14, 13, 14, 15,  2, 18,  2,  7, 19,  3, 15, 20, 10, 19, 20,  6,
        4, 12, 13,  2, 20,  5,  7,  9,  6, 16, 11, 15, 14])}
1
strain_number:  0 14.333333333333334
strain_various:  0 6.262764742685312
strain_number:  1 11.4
strain_various:  1 5.782732917920384
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 9, 17, 16,  3, 13, 11, 20, 15, 13,  2, 17, 19, 12, 11, 14, 17,  1,
       16, 12, 17, 16, 16,  4,  6, 16,  9,  5, 10, 14, 17]), 1: array([11, 14, 13, 14, 15,  2, 18,  2,  7, 19,  3, 15, 20, 10, 19, 20,  6,
        4, 12, 13,  2, 20,  5,  7,  9,  6, 16, 11, 15, 14])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -29.838604428119176
glc__D_e_1:  -1.4376923308043246
glc__D_e_1:  -24.19635433751521
glc__D_e_1:  -23.8009730155645
glc__D_e_1:  -1.4814477667721277
glc__D_e_1:  -9.992097962767366
strain_number:  0 43.766666666666666
strain_various:  0 15.982664219571014
strain_number:  1 2.466666666666667
strain_various:  1 1.0873004286866728
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 804.0127712200078, 314.48957949944787, 371.12364332004444, 461.95298166745533, 508.2972967750981, 470.19284741981613, 477.8159902103724, 452.9451839656558, 473.4610519365962, 427.19064637776944, 338.0796642989329, 362.76542261283146]
slip_r:  362.76542261283146
14
glc__D_e_1:  -17.793759942597035
glc__D_e_1:  -1.5556215423271382
glc__D_e_1:  -0.8362999202710863
glc__D_e_1:  -14.100438831559343
glc__D_e_1:  -22.605438964314427
glc__D_e_1:  -1.8062604276526724
glc__D_e_1:  -0.532841933504656
glc__D_e_1:  -16.398714405446007
glc__D_e_1:  -21.330475186576546
glc__D_e_1:  -0.06253270159178226
glc__D_e_1:  -8.643

glc__D_e_1:  -4.831515658309934
glc__D_e_1:  -4.738663979755949
glc__D_e_1:  -1.9692484783294357
glc__D_e_1:  -4.505541513141832
glc__D_e_1:  -6.26337706370124
strain_number:  0 4.766666666666667
strain_various:  0 2.417758374105145
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 804.0127712200078, 314.48957949944787, 371.12364332004444, 461.95298166745533, 508.2972967750981, 470.19284741981613, 477.8159902103724, 452.9451839656558, 473.4610519365962, 427.19064637776944, 338.0796642989329, 362.76542261283146, 375.5139989771948, 415.67675821549335, 347.683047416882, 221.42498794896497, 197.9383116883117]
slip_r:  197.9383116883117
19
glc__D_e_1:  -1.6018895871185146
glc__D_e_1:  -0.15052983559561484
glc__D_e_1:  -0.15163736845654507
glc__D_e_1:  -0.9170429071544126
glc__D_e_1:  -0.24091770454692862
glc__D_e_1:  -0.29313269199942804
glc__D_e_1:  -1.0172993032622828
glc__D_e_1:  -0.7726680440578236
glc__D_e_1:  -0.0068354034222281035
glc__D_e_1:  -1

In [10]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8, 19, 13,  3,  1, 19, 14, 18,  6,  9,  2, 11,  9,  1, 12,  5, 20,
       12,  5,  4,  7, 17,  6, 13,  8, 12, 16,  5, 15, 10]), 1: array([14, 17, 13,  8, 18, 15,  3,  9,  5,  5, 10, 14,  6,  5,  4,  6,  8,
        5,  4,  8, 16,  6,  5,  2,  4, 17,  3, 17,  5, 18])}
1
strain_number:  0 11.6
strain_various:  0 6.590902821313633
strain_number:  1 9.0
strain_various:  1 5.253570214625479
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 8, 19, 13,  3,  1, 19, 14, 18,  6,  9,  2, 11,  9,  1, 12,  5, 20,
       12,  5,  4,  7, 17,  6, 13,  8, 12, 16,  5, 15, 10]), 1: array([14, 17, 13,  8, 18, 15,  3,  9,  5,  5, 10, 14,  6,  5,  4,  6,  8,
        5,  4,  8, 16,  6,  5,  2,  4, 17,  3, 17,  5, 18])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404,

glc__D_e_1:  -17.864272019033354
glc__D_e_1:  -1.6261336187634576
glc__D_e_1:  -0.3064317432593773
glc__D_e_1:  -14.611264078808873
glc__D_e_1:  -29.653250049907626
glc__D_e_1:  -0.5836639041602645
glc__D_e_1:  -10.65441590700405
glc__D_e_1:  -20.526339648550746
glc__D_e_1:  -21.491769958225042
glc__D_e_1:  -6.405025829886746
glc__D_e_1:  -18.415566660034283
glc__D_e_1:  -21.595324526048913
glc__D_e_1:  -23.50661036757675
glc__D_e_1:  -32.98982929187211
glc__D_e_1:  -23.881657512227832
glc__D_e_1:  -20.163831111578485
glc__D_e_1:  -0.4396285002333026
glc__D_e_1:  -9.540993714476606
glc__D_e_1:  -29.075719515770515
glc__D_e_1:  -9.245479630573895
glc__D_e_1:  -21.22643077696339
glc__D_e_1:  -1.4093882495602903
glc__D_e_1:  -16.234567297240403
strain_number:  0 44.4
strain_various:  0 21.487360625881127
strain_number:  1 0.36666666666666664
strain_various:  1 0.6574360974438673
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 690.8362356812435, 264.4668847664204, 359.15914348809093, 429.952016

glc__D_e_1:  -0.7161187304016572
strain_number:  0 5.0
strain_various:  0 3.3862466931200785
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 690.8362356812435, 264.4668847664204, 359.15914348809093, 429.95201670104933, 493.3568014587752, 477.0033712546483, 384.4106176073669, 417.2767515331269, 384.40786837895445, 358.69840067985484, 413.4711180812942, 394.72924846045464, 326.5468437852499, 296.4370914398811, 249.7303058238295, 214.43508576643939, 206.03514668360367, 209.081199597504, 183.77882205513785]
slip_r:  183.77882205513785
21
glc__D_e_1:  -0.08029891179274062
glc__D_e_1:  -0.7878877974508662
glc__D_e_1:  -0.5091758543603141
glc__D_e_1:  -2.5548464135247024
glc__D_e_1:  -2.23510452696995
glc__D_e_1:  -1.588793350538471
glc__D_e_1:  -6.386198603181825
glc__D_e_1:  -2.4733455952072694
glc__D_e_1:  -3.265072287104318
glc__D_e_1:  -4.926797562442657
glc__D_e_1:  -3.8422960287787515
glc__D_e_1:  -3.8187706254188694
glc__D_e_1:  -3.4747735065591

In [11]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([19, 18, 19, 17,  8,  5, 12, 19, 17,  5,  9, 18, 14,  6, 20, 15,  5,
       15, 15, 12,  3, 14, 17,  4,  1, 10, 19, 14,  7,  8]), 1: array([ 8, 16,  4, 14, 13,  5,  7, 20,  4, 13, 19,  6,  7,  8, 19,  8, 13,
       10, 20, 16, 10, 10, 13, 16, 10,  9,  4, 11,  3,  6])}
1
strain_number:  0 14.166666666666666
strain_various:  0 6.728463585561135
strain_number:  1 10.733333333333333
strain_various:  1 5.0062183554278
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([19, 18, 19, 17,  8,  5, 12, 19, 17,  5,  9, 18, 14,  6, 20, 15,  5,
       15, 15, 12,  3, 14, 17,  4,  1, 10, 19, 14,  7,  8]), 1: array([ 8, 16,  4, 14, 13,  5,  7, 20,  4, 13, 19,  6,  7,  8, 19,  8, 13,
       10, 20, 16, 10, 10, 13, 16, 10,  9,  4, 11,  3,  6])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639

glc__D_e_1:  -50.33743902995029
glc__D_e_1:  -22.332630887915975
glc__D_e_1:  -1.5334523512542209
glc__D_e_1:  -1.8896359800384723
glc__D_e_1:  -25.04036242180849
glc__D_e_1:  -26.986136015883766
glc__D_e_1:  -2.2945913912626636
glc__D_e_1:  -1.6186855677584657
glc__D_e_1:  -17.484558039699817
glc__D_e_1:  -12.695564670062492
glc__D_e_1:  -1.0184664061844515
glc__D_e_1:  -22.49662863024561
glc__D_e_1:  -15.248011164999035
glc__D_e_1:  -1.1988935252922666
glc__D_e_1:  -7.67965688907746
glc__D_e_1:  -1.7850078300563368
strain_number:  0 43.0
strain_various:  0 17.80636590286369
strain_number:  1 2.4
strain_various:  1 1.6041612554021285
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 584.728616777959, 323.94617224880386, 387.24071177360645, 443.6906677198486, 415.52709732966525, 393.3206240348643, 398.74345158524267, 404.9212609552175, 435.74243313056644, 366.50249753411316, 344.86957256627386, 348.7501811106044]
slip_r:  348.7501811106044
14
glc__D_e_1:  -11.443792306038858
glc__D_e_1:  -5.2

glc__D_e_1:  -2.782413344092497
glc__D_e_1:  -7.339828272057653
glc__D_e_1:  -6.042285216872226
glc__D_e_1:  -6.5861268576794325
glc__D_e_1:  -14.017753130070272
glc__D_e_1:  -5.109585998315621
glc__D_e_1:  -12.083268977389235
glc__D_e_1:  -12.815275451661726
glc__D_e_1:  -4.255167737696499
strain_number:  0 6.033333333333333
strain_various:  0 4.976500332116492
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 584.728616777959, 323.94617224880386, 387.24071177360645, 443.6906677198486, 415.52709732966525, 393.3206240348643, 398.74345158524267, 404.9212609552175, 435.74243313056644, 366.50249753411316, 344.86957256627386, 348.7501811106044, 385.4993713867949, 348.4113585210164, 355.63639125748887, 238.77779421834347, 217.75682418716474]
slip_r:  217.75682418716474
19
glc__D_e_1:  -0.17045101381298589
glc__D_e_1:  -1.370685553030741
glc__D_e_1:  -1.3289948519699426
glc__D_e_1:  -1.4631105540416802
glc__D_e_1:  -2.2160801820382012
glc__D_e_1:  -0.265

In [12]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([18,  6, 13, 10,  8, 19, 11, 16,  5,  8, 12, 12, 14, 15, 11, 12, 11,
        9,  2,  1,  1,  3, 13, 12, 16, 18,  7, 17, 13, 16]), 1: array([ 4,  8,  8,  1,  8,  4, 15,  4,  1,  7, 13, 17, 14,  7,  4, 14, 10,
        2, 19, 19,  6, 11,  4, 17,  2,  7, 10,  4,  5,  4])}
1
strain_number:  0 12.766666666666667
strain_various:  0 5.987115796070388
strain_number:  1 8.3
strain_various:  1 5.361280941466632
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([18,  6, 13, 10,  8, 19, 11, 16,  5,  8, 12, 12, 14, 15, 11, 12, 11,
        9,  2,  1,  1,  3, 13, 12, 16, 18,  7, 17, 13, 16]), 1: array([ 4,  8,  8,  1,  8,  4, 15,  4,  1,  7, 13, 17, 14,  7,  4, 14, 10,
        2, 19, 19,  6, 11,  4, 17,  2,  7, 10,  4,  5,  4])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988

glc__D_e_1:  -22.83600926303955
glc__D_e_1:  -23.216138285499714
glc__D_e_1:  -29.18731083703575
glc__D_e_1:  -27.31765175738414
glc__D_e_1:  -14.78722147504639
glc__D_e_1:  -49.98364735999487
glc__D_e_1:  -12.693907217725267
glc__D_e_1:  -28.130318872797677
glc__D_e_1:  -38.27679123996708
glc__D_e_1:  -22.40254739994577
glc__D_e_1:  -32.22090085058763
glc__D_e_1:  -24.282029562446347
glc__D_e_1:  -31.284489869891058
glc__D_e_1:  -13.167127076086652
glc__D_e_1:  -20.867736551583114
glc__D_e_1:  -5.4365319129407865
glc__D_e_1:  -8.458096475949617
glc__D_e_1:  -17.673543528000405
glc__D_e_1:  -21.178574875908314
glc__D_e_1:  -19.236003743061886
glc__D_e_1:  -10.070602557288675
glc__D_e_1:  -14.642401500399371
glc__D_e_1:  -14.671774323000548
glc__D_e_1:  -15.026746538121122
strain_number:  0 37.8
strain_various:  0 18.227086071741326
strain_number:  1 0.06666666666666667
strain_various:  1 0.24944382578492943
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 610.8994197237231, 404.3995051001629

glc__D_e_1:  -0.20891938394319354
glc__D_e_1:  -0.16912503423505632
glc__D_e_1:  -0.3742194599725224
glc__D_e_1:  -0.5047876150814814
glc__D_e_1:  -0.34500586232555874
glc__D_e_1:  -0.561281025743206
glc__D_e_1:  -0.26026176578368165
glc__D_e_1:  -0.8988207753327944
glc__D_e_1:  -1.706761894618965
glc__D_e_1:  -0.503400454047472
glc__D_e_1:  -1.408612316800229
glc__D_e_1:  -1.3260933527415346
glc__D_e_1:  -1.2894200221166776
glc__D_e_1:  -1.8080559909730236
glc__D_e_1:  -6.385407675265931
glc__D_e_1:  -7.781699073339573
glc__D_e_1:  -7.347601546317802
glc__D_e_1:  -4.016261336347198
glc__D_e_1:  -3.9211928521821338
glc__D_e_1:  -2.266839094175954
strain_number:  0 2.8333333333333335
strain_various:  0 2.7938424357067015
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 610.8994197237231, 404.39950510016297, 356.8133664581033, 369.93091250854417, 438.92483351759665, 473.85137300197727, 507.9022893772894, 441.8115367529727, 412.21251930529013, 453.40

In [6]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2,  6, 18, 19,  2,  1, 18, 11, 17,  3, 14, 11, 12,  6,  9, 16, 18,
       20,  5,  4,  9, 16, 16,  3, 16, 12,  9,  5,  2,  3]), 1: array([13,  6, 12,  3, 14,  2, 14,  6,  4,  6, 12,  1, 15,  9, 12, 17,  7,
       12,  3,  9, 16,  2,  6,  4,  7, 14, 19,  8, 12, 16])}
1
strain_number:  0 11.7
strain_various:  0 7.37631344236401
strain_number:  1 9.366666666666667
strain_various:  1 5.029800084385948
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2,  6, 18, 19,  2,  1, 18, 11, 17,  3, 14, 11, 12,  6,  9, 16, 18,
       20,  5,  4,  9, 16, 16,  3, 16, 12,  9,  5,  2,  3]), 1: array([13,  6, 12,  3, 14,  2, 14,  6,  4,  6, 12,  1, 15,  9, 12, 17,  7,
       12,  3,  9, 16,  2,  6,  4,  7, 14, 19,  8, 12, 16])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841

glc__D_e_1:  -26.89344256608557
glc__D_e_1:  -28.084790912759967
glc__D_e_1:  -25.119051467111177
glc__D_e_1:  -10.066768090800148
glc__D_e_1:  -0.7130787395542199
glc__D_e_1:  -24.257103088767266
glc__D_e_1:  -7.51716819428797
glc__D_e_1:  -18.325678937061515
glc__D_e_1:  -18.233428397414585
glc__D_e_1:  -16.315243533745925
glc__D_e_1:  -24.057789464703248
glc__D_e_1:  -0.4400080589497275
glc__D_e_1:  -8.89283451461905
glc__D_e_1:  -26.28029225296398
glc__D_e_1:  -0.3494051248751547
glc__D_e_1:  -22.33971907642832
glc__D_e_1:  -25.453689959040748
glc__D_e_1:  -15.428996944200067
glc__D_e_1:  -18.3451849551583
glc__D_e_1:  -16.253016647151565
glc__D_e_1:  -4.899973363203689
strain_number:  0 39.43333333333333
strain_various:  0 19.181038090317795
strain_number:  1 0.4666666666666667
strain_various:  1 1.0873004286866725
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 814.911378675504, 338.0767883432357, 356.12197670750305, 455.7304837706154, 427.5347486807777, 431.89628946086924, 465.510111

glc__D_e_1:  -4.738392986545755
glc__D_e_1:  -4.951114956397203
glc__D_e_1:  -4.569610165869081
glc__D_e_1:  -1.9443495229376608
glc__D_e_1:  -2.114188000125431
glc__D_e_1:  -3.7264679537208676
glc__D_e_1:  -0.44961383060631577
glc__D_e_1:  -0.44685899891356395
glc__D_e_1:  -0.22173884694277768
glc__D_e_1:  -0.7379706498747443
glc__D_e_1:  -0.28592453051423544
glc__D_e_1:  -0.3992312379560521
glc__D_e_1:  -0.07856825499194509
glc__D_e_1:  -0.15578771062257157
glc__D_e_1:  -0.45125665842682006
glc__D_e_1:  -0.2172613343866575
glc__D_e_1:  -0.5601719454278029
glc__D_e_1:  -1.1011574960500123
glc__D_e_1:  -1.2485869511439842
glc__D_e_1:  -2.71010222606128
glc__D_e_1:  -3.253342944879739
glc__D_e_1:  -4.387560827093897
glc__D_e_1:  -4.438024073844309
glc__D_e_1:  -15.81723034039541
glc__D_e_1:  -15.282859119263522
strain_number:  0 3.933333333333333
strain_various:  0 5.039400317057136
strain_number:  1 0.03333333333333333
strain_various:  1 0.17950549357115014
fd_r_threshold:  [1.1, 1.1, 

In [7]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 4, 14, 18, 10, 17,  7,  7, 11, 20,  9, 11,  1, 16,  6,  8, 16,  2,
       10,  8, 18, 13, 17, 16, 12,  2,  4, 10,  7,  2, 13]), 1: array([17,  4, 20,  6, 14, 12, 14,  8, 17,  8, 14,  5, 16, 10,  6, 10, 15,
        8,  8, 18,  8,  3,  2, 14,  6, 18,  7, 16, 20, 18])}
1
strain_number:  0 11.966666666666667
strain_various:  0 6.493501024528722
strain_number:  1 11.4
strain_various:  1 5.314132102234569
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 4, 14, 18, 10, 17,  7,  7, 11, 20,  9, 11,  1, 16,  6,  8, 16,  2,
       10,  8, 18, 13, 17, 16, 12,  2,  4, 10,  7,  2, 13]), 1: array([17,  4, 20,  6, 14, 12, 14,  8, 17,  8, 14,  5, 16, 10,  6, 10, 15,
        8,  8, 18,  8,  3,  2, 14,  6, 18,  7, 16, 20, 18])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.50901728

glc__D_e_1:  -14.96680686840088
glc__D_e_1:  -8.660733078992813
glc__D_e_1:  -2.228008477403094
glc__D_e_1:  -10.104361646076104
glc__D_e_1:  -7.155029653297817
glc__D_e_1:  -1.5715344691622137
glc__D_e_1:  -15.307360661619404
glc__D_e_1:  -6.953332163279065
glc__D_e_1:  -0.03248888227844038
glc__D_e_1:  -1.6259366125171688
glc__D_e_1:  -6.593244833976868
strain_number:  0 42.53333333333333
strain_various:  0 15.27248797311325
strain_number:  1 2.3666666666666667
strain_various:  1 1.7792008193443356
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 728.4056875786754, 337.7970903580424, 324.5406994321468, 370.2883320626742, 436.35409015108524, 426.74513688495887, 453.0574472112035, 418.4572313463761, 408.2835152427504, 353.63375776004636, 396.3845043635695, 306.53313810475777, 417.2067338213855]
slip_r:  417.2067338213855
15
glc__D_e_1:  -15.712161994383258
glc__D_e_1:  -0.3525719664953826
glc__D_e_1:  -0.22419930486832707
glc__D_e_1:  -11.856443938688953
glc__D_e_1:  -21.130636732512194
glc_

glc__D_e_1:  -10.926726016060565
glc__D_e_1:  -23.233412764913556
glc__D_e_1:  -0.2843054075925462
glc__D_e_1:  -0.20224567381324293
glc__D_e_1:  -6.137552182880972
strain_number:  0 6.566666666666666
strain_various:  0 6.151874149846984
strain_number:  1 0.03333333333333333
strain_various:  1 0.17950549357115014
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 728.4056875786754, 337.7970903580424, 324.5406994321468, 370.2883320626742, 436.35409015108524, 426.74513688495887, 453.0574472112035, 418.4572313463761, 408.2835152427504, 353.63375776004636, 396.3845043635695, 306.53313810475777, 417.2067338213855, 385.00272072735186, 349.3694005706735, 287.5596800234533, 223.72049264449828, 243.30212287344085]
slip_r:  243.30212287344085
20
glc__D_e_1:  -0.5893446387054397
glc__D_e_1:  -0.17153909424757208
glc__D_e_1:  -0.1299686882690324
glc__D_e_1:  -1.0265865975971622
glc__D_e_1:  -0.9106069829509403
glc__D_e_1:  -0.973862835382376
glc__D_e_1:  -0.5776941845501855
glc__D_e_1:  -0.441014297161032

In [8]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([11, 19,  2, 13, 17,  4, 16, 11,  9, 11, 20, 17,  6, 11, 19, 18,  1,
       18, 16,  9, 18,  3,  1,  7, 20, 11, 17, 20, 20, 14]), 1: array([ 5, 17,  8, 19,  5, 20, 14,  8, 12, 16, 14,  1, 17,  1,  2, 18,  3,
       12,  3,  9,  2,  3, 11, 14,  6, 19, 15,  6, 15, 16])}
1
strain_number:  0 14.766666666666667
strain_various:  0 7.40577852460871
strain_number:  1 10.366666666666667
strain_various:  1 6.129074608418106
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([11, 19,  2, 13, 17,  4, 16, 11,  9, 11, 20, 17,  6, 11, 19, 18,  1,
       18, 16,  9, 18,  3,  1,  7, 20, 11, 17, 20, 20, 14]), 1: array([ 5, 17,  8, 19,  5, 20, 14,  8, 12, 16, 14,  1, 17,  1,  2, 18,  3,
       12,  3,  9,  2,  3, 11, 14,  6, 19, 15,  6, 15, 16])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5

glc__D_e_1:  -23.977260211399525
glc__D_e_1:  -3.034174999376047
glc__D_e_1:  -0.8762257492770291
glc__D_e_1:  -16.651698110190672
glc__D_e_1:  -13.515544551562893
glc__D_e_1:  -1.1917614554482725
glc__D_e_1:  -1.5422443533904402
glc__D_e_1:  -27.675786032036623
glc__D_e_1:  -17.89806074787601
glc__D_e_1:  -1.0205672541015076
glc__D_e_1:  -0.7062148047636669
glc__D_e_1:  -14.927976768017428
glc__D_e_1:  -24.51740802917983
glc__D_e_1:  -0.05039720597219244
glc__D_e_1:  -0.5298692044364872
glc__D_e_1:  -8.536789577050724
glc__D_e_1:  -19.57507214290024
glc__D_e_1:  -1.1796751832391106
glc__D_e_1:  -0.3950168718608573
glc__D_e_1:  -13.580971903341364
glc__D_e_1:  -27.393599635460667
glc__D_e_1:  -11.21028669244167
glc__D_e_1:  -17.408510709226373
glc__D_e_1:  -21.990729975959905
glc__D_e_1:  -44.7692188228599
glc__D_e_1:  -27.514490490533895
glc__D_e_1:  -22.441772047979963
glc__D_e_1:  -25.992184151612996
glc__D_e_1:  -29.709092919684267
glc__D_e_1:  -19.038903846419622
glc__D_e_1:  -32.

glc__D_e_1:  -1.142416956289278
glc__D_e_1:  -3.5708390116574176
glc__D_e_1:  -0.30519304980011885
glc__D_e_1:  -1.5241346838687078
glc__D_e_1:  -2.063785070136084
glc__D_e_1:  -2.830462059253321
glc__D_e_1:  -3.4314550806495188
glc__D_e_1:  -4.449374342307063
glc__D_e_1:  -4.306658867485731
glc__D_e_1:  -2.557856347640109
glc__D_e_1:  -1.9019260139183158
glc__D_e_1:  -2.658427593698108
glc__D_e_1:  -0.19085691822860051
glc__D_e_1:  -0.8519280356904384
strain_number:  0 3.033333333333333
strain_various:  0 2.008039397577204
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 920.4082124660682, 371.60469319870845, 453.7687775588705, 408.233670569197, 338.16377334755543, 333.4153480702514, 381.3010428329998, 444.43810685438103, 429.1696291453111, 469.5522662908965, 320.1745797223638, 307.826591075612, 355.9324455897946, 268.81581770599007, 331.4066687868933, 179.57691851159595, 175.4601648351648, 206.18777056277057]
slip_r:  206.18777056277057
20
glc__

In [9]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10, 12, 10,  5, 15, 17,  9,  6,  6, 15, 19,  1,  4,  9,  3,  1,  5,
       18,  5,  3,  2, 14,  5, 16, 14, 11, 16,  8,  5, 11]), 1: array([16,  6,  6, 15, 18, 11,  4,  7,  7, 11, 17, 20, 13,  3, 14, 13, 20,
        4,  1, 17,  5,  3, 18, 15,  2, 16, 13,  1, 17, 20])}
1
strain_number:  0 10.666666666666666
strain_various:  0 6.363087999461338
strain_number:  1 11.1
strain_various:  1 6.262853449772981
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10, 12, 10,  5, 15, 17,  9,  6,  6, 15, 19,  1,  4,  9,  3,  1,  5,
       18,  5,  3,  2, 14,  5, 16, 14, 11, 16,  8,  5, 11]), 1: array([16,  6,  6, 15, 18, 11,  4,  7,  7, 11, 17, 20, 13,  3, 14, 13, 20,
        4,  1, 17,  5,  3, 18, 15,  2, 16, 13,  1, 17, 20])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.50901728

glc__D_e_1:  -0.6941819089620758
glc__D_e_1:  -0.9596183554976588
glc__D_e_1:  -14.145573386978166
glc__D_e_1:  -23.62798262871374
glc__D_e_1:  -0.678875271392732
glc__D_e_1:  -0.5854416670078152
glc__D_e_1:  -11.699782834941814
glc__D_e_1:  -27.28446188452158
glc__D_e_1:  -0.22645718510708535
glc__D_e_1:  -18.59144687545386
glc__D_e_1:  -17.06239535761011
glc__D_e_1:  -11.15731687741333
glc__D_e_1:  -6.43489190184874
glc__D_e_1:  -5.702478215989981
glc__D_e_1:  -13.607898499803962
glc__D_e_1:  -25.770192440326788
glc__D_e_1:  -12.395279523103472
glc__D_e_1:  -20.132130883344008
glc__D_e_1:  -0.8875045062287619
glc__D_e_1:  -1.5442882551029977
glc__D_e_1:  -18.87347101367652
glc__D_e_1:  -12.206032034932996
glc__D_e_1:  -2.5129765266924067
glc__D_e_1:  -11.555703831079896
strain_number:  0 37.4
strain_various:  0 15.315787062156922
strain_number:  1 1.1
strain_various:  1 0.9433981132056604
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 875.9880571569813, 338.7281369945845, 380.04307205951

glc__D_e_1:  -0.5225076520518577
glc__D_e_1:  -0.3934125181158299
glc__D_e_1:  -1.8897456080413546
glc__D_e_1:  -0.016181722209990257
glc__D_e_1:  -0.6193874758312612
glc__D_e_1:  -2.464173678115534
glc__D_e_1:  -1.2902821444835826
glc__D_e_1:  -4.5039624431489855
glc__D_e_1:  -8.424317167381163
glc__D_e_1:  -4.496733960321104
glc__D_e_1:  -13.814328783278892
glc__D_e_1:  -11.584831785337476
glc__D_e_1:  -23.350165519839383
glc__D_e_1:  -19.030900966400697
glc__D_e_1:  -7.276802384496381
glc__D_e_1:  -2.7782514188121854
glc__D_e_1:  -7.110877177853
glc__D_e_1:  -3.5889687607274636
glc__D_e_1:  -0.7612165478601415
glc__D_e_1:  -1.3674088029012657
glc__D_e_1:  -0.32760225642350077
glc__D_e_1:  -0.15419191822032763
glc__D_e_1:  -0.06769945599435312
glc__D_e_1:  -1.1642507024599158
glc__D_e_1:  -0.09411132724092308
strain_number:  0 6.0
strain_various:  0 7.737355275975549
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 875.9880571569813, 338.7281369